In [ ]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install librosa
!pip install scikit-learn
!pip install torch
!pip install datasets
!pip install torchaudio

In [ ]:
from transformers import ASTModel, AutoProcessor
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import os
import librosa
from sklearn import svm
from sklearn.model_selection import train_test_split
from transformers import AutoProcessor, ASTModel
from transformers import ASTFeatureExtractor
import torch
from datasets import load_dataset
import torchaudio

Загрузим датасет

In [ ]:
path_audio_train = './audio_train/train'
dataset = load_dataset(path_audio_train)  #[5683]

In [ ]:
print(dataset['train'][0]['audio']['path'][-24:])

Воспользуемся претренированной моделью и жкстрактором фичей

In [ ]:
model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [ ]:
processor = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

Извлечем фичи

In [ ]:
inputs = processor(dataset['train'][0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

In [ ]:
inputs_of_every_audio = []

In [ ]:
for i in range(len(dataset['train'])):
    inputs = processor(dataset['train'][i]["audio"]["array"], sampling_rate=16000, return_tensors="pt")
    inputs_of_every_audio.append(inputs)

Скормим модели фичи, полученные с помощью экстрактора фичей

In [ ]:
outputs_of_every_audio = []

In [ ]:
for i in range(len(inputs_of_every_audio)):
    with torch.no_grad():
        outputs = model(**inputs_of_every_audio[i])
    outputs_of_every_audio.append(outputs)

Возьмем то часть обработанных данных, которая понадобится позже

In [ ]:
pooler_output_of_every_audio = []
for i in range(len(dataset['train'])):
    pooler_output = outputs_of_every_audio[i].pooler_output[0].tolist()
    pooler_output_of_every_audio.append(pooler_output)

In [ ]:
audios_features = []
audios_names = []

Запишем массив названий файлов и массив фичей

In [ ]:
for i in range(len(dataset['train'])):
    audios_features.append(pooler_output_of_every_audio[i])
    audios_names.append(dataset['train'][i]['audio']['path'][-24:])

In [ ]:
print(audios_names[0])

In [ ]:
data = {'fname': audios_names, 'features': audios_features}

In [ ]:
data_features = pd.DataFrame(data)

In [ ]:
print(data_features.head())

In [ ]:
data_features.to_csv('features_of_train.csv')

Поработаем с классами

In [ ]:
data_classes = pd.read_csv('train.csv')
data_classes.sort_values(by='fname', inplace=True)
data_classes = data_classes.reset_index()
del data_classes['index']
data_classes.head()

In [ ]:
classes_labels = {}
reverse_classes_labels = {}
for i in range(len(classes)):
    classes_labels[classes[i]] = i
    reverse_classes_labels[i] = classes[i]

In [ ]:
def process_label(label):
    return classes_labels[label]

In [ ]:
data_classes.label = data_classes.label.apply(process_label)

In [ ]:
data_classes.head(5)

In [ ]:
features = []
for i in range(len(data_features.features)):
    features.append(data_features.features[i])

In [ ]:
print(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    data_classes.label, test_size=0.2, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [ ]:
print(X_train.var())

Используем svm для классификации аудио по фичам, которые получили с помощью AST

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', C=3, gamma=1/(768*X_train.var()), kernel='rbf')

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score

Смотрим на f1-score и пытаемся повысить

In [ ]:
f1_score(y_test, y_pred, average='macro')

0.8454958147097228 - poly test=0.15

0.8479759748885759 - rbf test=0.15

0.8538223912214986 - poly test=0.15

Далее делаем предсказание

In [ ]:
path_audio_train = './audio_test/test'
dataset = load_dataset(path_audio_train)  #[5683]

In [ ]:
sampling_rate = 16000

In [ ]:
inputs_of_every_test_audio = []

In [ ]:
for i in range(len(dataset['train'])):
    inputs = processor(dataset['train'][i]["audio"]["array"], sampling_rate=16000, return_tensors="pt")
    inputs_of_every_test_audio.append(inputs)

In [ ]:
print(len(inputs_of_every_test_audio))
print(inputs_of_every_test_audio[0])

In [ ]:
outputs_of_every_test_audio = []

In [ ]:
for i in range(len(inputs_of_every_test_audio)):
    with torch.no_grad():
        outputs = model(**inputs_of_every_test_audio[i])
    outputs_of_every_audio.append(outputs)

In [ ]:
outputs_of_every_test_audio = outputs_of_every_audio[5683:]

In [ ]:
outputs_of_every_test_audio[0]

In [ ]:
pooler_output_of_every_test_audio = []
for i in range(len(dataset['train'])):
    pooler_output = outputs_of_every_test_audio[i].pooler_output[0].tolist()
    pooler_output_of_every_test_audio.append(pooler_output)

In [ ]:
test_audios_features = []
test_audios_names = []

In [ ]:
for i in range(len(dataset['train'])):
    test_audios_features.append(pooler_output_of_every_test_audio[i])
    test_audios_names.append(dataset['train'][i]['audio']['path'][-24:])

In [ ]:
test_data = {'fname': test_audios_names, 'features': test_audios_features}

In [ ]:
test_data_features = pd.DataFrame(test_data)

In [ ]:
print(test_data_features.head())

In [ ]:
test_data_features.to_csv('features_of_test_then_train.csv', index=False)

In [ ]:
not_index_test_data_features = test_data_features.reset_index()
del not_index_test_data_features['index']

In [ ]:
# print(list(not_index_test_data_features.features.head()))

In [ ]:
y_test_pred = clf.predict(list(test_data_features.features))

In [ ]:
y_test_pred[:10]

In [ ]:
audios_test_classes = []
for i in range(len(dataset['train'])):
    audios_test_classes.append(reverse_classes_labels[y_test_pred[i]])

In [ ]:
print(len(audios_test_classes))

In [ ]:
submission = pd.DataFrame({'fname': test_audios_names, 'label': audios_test_classes})

In [ ]:
print(submission.tail())

In [ ]:
submission.to_csv('submission_AST_model.csv', header=True, index=False)